# ¿Cómo afecta la localización al éxito de tu negocio?

## Tabla de contenidos
* [Problema comercial](#problema)
* [Datos](#datos)
* [Metodología](#metodología)
* [Análisis](#análisis)
* [Resultados y discusión](#resultados)
* [Conclusión](#conclusión)

## Problema comercial <a name="problema"></a>

La localización de una empresa es un elemento clave para su correcto desarrollo, y puede constituir uno de los **factores competitivos** más importantes de la misma. En este sentido, una empresa podría decidir colocarse cerca de sus competidores si no puede competir en precios, ya que vende un producto homogéneo, o podría colocarse a cierta distancia si esto le otorga cierto poder de mercado.

Por este motivo, los dueños de los restaurantes estarían interesados en acceder a un servicio que pudiese aportar información adicional sobre la localización de los otros restaurantes en la ciudad, de forma que puedan tomar una decisión informada sobre este factor clave. El objetivo de este análisis es, por tanto, **predecir el éxito de un restaurante en función de su localización**.

Para ello, voy a diseñar un **sistema de recomendación colaborativo** que permita elaborar una lista para un barrio de la ciudad de Los Ángeles que contenga qué restaurantes tendrían más éxito de abrir uno ahí, en función de la **categoría del restaurante**.

## Datos <a name="datos"></a>

Para poder abordar este problema, no solamente es necesario obtener datos sobre la localización de los diferentes restaurantes de una ciudad, sino que también es necesario adquirir **datos que permitan medir el éxito de los restaurantes**. Precisamente por este motivo, además de emplear los datos de la **API de Foursquare** (para la ciudad de Los Ángeles, California), utilizaremos también datos de **reseñas de Google** que los usuarios dejan en estos negocios, mediante el uso de la **API de Google Maps**. Estos datos de reseña tienen un valor de **1 a 5 estrellas**, donde 1 sería la puntuación más baja que se puede otorgar a un negocio.

La API de Foursquare necesita los datos de *latitud y longitud* de los barrios objetivo. Para obtener esta información, empleamos la **API de Google Geocode**. Una vez hecho esto, se pueden obtener los datos de los diferentes negocios que existen en cada barrio a través de la API de Foursquare y, finalmente, se pueden usar sus nombres para obtener las reseñas que los usuarios de Google hicieron sobre el negocio. 

### Barrios

Primeramente, es necesario obtener los datos sobre los diferentes vecindarios de la ciudad de California, de forma que se pueda establecer un área de interés. Para ello, es necesario obtener los datos de latitud y longitud. Veamos cómo se realiza este proceso.  

Primeramente, es necesario obtener una lista con todos los vecindarios, para lo que emplearemos la página de Wikipedia correspondiente y el paquete **BeautifulSoup**.

In [2]:
soup = BeautifulSoup(html_doc, 'html.parser')
places = []
for link in soup.find_all('a'):
    if "#" in link.get('href') or "index" in link.get('href'):
        pass
    else:
        places.append(link.get('href')[link.get('href').rfind("/")+1:link.get('href').find(",")].replace("_"," "))

In [3]:
import requests
def get_latlng(address):
    try:
        url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(address,API_key)
        results = requests.get(url).json()
        geo_data = results["results"][0]["geometry"]["location"]
        lat,lng = (geo_data["lat"],geo_data["lng"])
        print("{}. Latitud {} y Longitud {}".format(address,lat,lng))
        return address,lat,lng
    except:
        pass

In [4]:
barrios = []
for i in places:
    barrios.append(get_latlng(i))

Adams-Normandie. Latitud 34.0343441 y Longitud -118.2993138
Alsace. Latitud 48.3181795 y Longitud 7.441624099999999
Angelino Heights. Latitud 34.0702889 y Longitud -118.2547965
Angeles Mesa. Latitud 33.9955656 y Longitud -118.3219894
Angelus Vista. Latitud 34.0471577 y Longitud -118.3177106
Arleta. Latitud 34.2504595 y Longitud -118.4338345
Arlington Heights. Latitud 42.0883603 y Longitud -87.98062650000001
Arts District. Latitud 34.0418947 y Longitud -118.2326448
Atwater Village. Latitud 34.1183176 y Longitud -118.2605846
Baldwin Hills. Latitud 34.0181993 y Longitud -118.3403506
Crenshaw. Latitud 34.0181993 y Longitud -118.3403506
Baldwin Village. Latitud 34.0150915 y Longitud -118.3476559
Baldwin Vista. Latitud 34.0134562 y Longitud -118.3627367
Beachwood Canyon. Latitud 34.1188801 y Longitud -118.3215322
Bel Air. Latitud 34.1002455 y Longitud -118.459463
Benedict Canyon. Latitud 34.0939284 y Longitud -118.4324553
Beverly Crest. Latitud 34.1012952 y Longitud -118.4162542
Beverly Glen

In [5]:
barrios = [i for i in barrios if i is not None]
barrios

[('Adams-Normandie', 34.0343441, -118.2993138),
 ('Alsace', 48.3181795, 7.441624099999999),
 ('Angelino Heights', 34.0702889, -118.2547965),
 ('Angeles Mesa', 33.9955656, -118.3219894),
 ('Angelus Vista', 34.0471577, -118.3177106),
 ('Arleta', 34.2504595, -118.4338345),
 ('Arlington Heights', 42.0883603, -87.98062650000001),
 ('Arts District', 34.0418947, -118.2326448),
 ('Atwater Village', 34.1183176, -118.2605846),
 ('Baldwin Hills', 34.0181993, -118.3403506),
 ('Crenshaw', 34.0181993, -118.3403506),
 ('Baldwin Village', 34.0150915, -118.3476559),
 ('Baldwin Vista', 34.0134562, -118.3627367),
 ('Beachwood Canyon', 34.1188801, -118.3215322),
 ('Bel Air', 34.1002455, -118.459463),
 ('Benedict Canyon', 34.0939284, -118.4324553),
 ('Beverly Crest', 34.1012952, -118.4162542),
 ('Beverly Glen', 34.1077162, -118.442596),
 ('Beverly Grove', 34.0734732, -118.3765717),
 ('Beverly Hills Post Offic', 34.0691217, -118.4067693),
 ('Beverly Park', 34.1177924, -118.4177707),
 ('Beverlywood', 34.0494

In [6]:
import pandas as pd
location_data = pd.DataFrame(data = barrios, columns = ["Neighborhood","Latitude","Longitude"])
location_data["Neighborhood"] = [i[:i.find(",")] for i in list(location_data["Neighborhood"].values)]
location_data.head()

,Neighborhood,Latitude,Longitude
0,Adams-Normandi,34.034344,-118.299314
1,Alsac,48.318179,7.441624
2,Angelino Height,34.070289,-118.254796
3,Angeles Mes,33.995566,-118.321989
4,Angelus Vist,34.047158,-118.317711


Una vez obtenidos los datos de latitud y longitud de los diferentes vecindarios, es interesante visualizarlos en el mapa.

In [7]:
import folium
la_map = folium.Map(location = [34.052235, -118.243683],zoom_start = 10)
for barrio in barrios:
    name,lat,long = barrio
    folium.Circle(location = [lat,long], popup = name, radius = 500, color = "blue", fill = True).add_to(la_map)

la_map

Ahora es el momento de usar estos datos para poder obtener los restaurantes localizados en los diferentes barrios de Los Ángeles. Para ello, empleamos la API de Foursquare.

In [8]:
import json
def getNearbyrestaurants(names, latitudes, longitudes, radius=500):
    
    restaurants_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&limit=50".format(lat, lng, radius)

        headers = {
            "Accept": "application/json",
            "Authorization": API_Key_FS
                }

        response = requests.get(url, headers=headers)
        results = json.loads(response.text)["results"]
        # regresa solo información relevante de cada sitio cercano
        restaurants_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['geocodes']['main']['latitude'], 
            v['geocodes']['main']['longitude'],  
            v['categories'][0]['name']) for v in results if len(v['categories'])>0 and len(v['geocodes'])>0])

    nearby_restaurants = pd.DataFrame([item for venue_list in restaurants_list for item in venue_list])
    nearby_restaurants.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_restaurants)

In [44]:
la_restaurants = getNearbyrestaurants(location_data["Neighborhood"], location_data["Latitude"], location_data["Longitude"])

Adams-Normandi 34.0343441 -118.2993138
Alsac 48.3181795 7.441624099999999
Angelino Height 34.0702889 -118.2547965
Angeles Mes 33.9955656 -118.3219894
Angelus Vist 34.0471577 -118.3177106
Arlet 34.2504595 -118.4338345
Arlington Height 42.0883603 -87.98062650000001
Arts Distric 34.0418947 -118.2326448
Atwater Villag 34.1183176 -118.2605846
Baldwin Hill 34.0181993 -118.3403506
Crensha 34.0181993 -118.3403506
Baldwin Villag 34.0150915 -118.3476559
Baldwin Vist 34.0134562 -118.3627367
Beachwood Canyo 34.1188801 -118.3215322
Bel Ai 34.1002455 -118.459463
Benedict Canyo 34.0939284 -118.4324553
Beverly Cres 34.1012952 -118.4162542
Beverly Gle 34.1077162 -118.442596
Beverly Grov 34.0734732 -118.3765717
Beverly Hills Post Offi 34.0691217 -118.4067693
Beverly Par 34.1177924 -118.4177707
Beverlywoo 34.0494132 -118.3952319
Boyle Height 34.0297895 -118.2117257
Brentwoo 34.0521011 -118.4732464
Brentwood Circl 34.0717832 -118.4710675
Brentwood Gle 34.0655045 -118.4626788
Brooksid 39.0222212 -94.585014

In [10]:
la_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandi,34.034344,-118.299314,Orange Door Sushi,34.032550,-118.299437,Sushi Restaurant
1,Adams-Normandi,34.034344,-118.299314,South Shore Barbers,34.033941,-118.298332,Health and Beauty Service
2,Adams-Normandi,34.034344,-118.299314,El Paseo,34.035435,-118.299110,Park
3,Adams-Normandi,34.034344,-118.299314,Master Works Painting,34.034762,-118.300045,Painter
4,Adams-Normandi,34.034344,-118.299314,Sushi Delight,34.032483,-118.299436,Sushi Restaurant


Una vez obtenidos estos datos, es necesario limpiarlos para obtener una base de datos donde solamente queden los restaurantes.

In [11]:
la_restaurants["Venue Category"].unique()

array(['Sushi Restaurant', 'Health and Beauty Service', 'Park', 'Painter',
       'Latin American Restaurant', 'Organization', 'Credit Union',
       'Fast Food Restaurant', 'Plumber', 'Automotive Repair Shop',
       'Equipment Rental Service', 'Shoe Store', 'Electronics Store',
       'Laundromat', 'Butcher', 'Ramen Restaurant',
       'Chemicals and Gasses Manufacturer', 'Repair Service',
       'Sports and Recreation', 'Hair Salon',
       'Business and Professional Services',
       'Business and Strategy Consulting Office',
       'Shipping, Freight, and Material Transportation Service', 'Garden',
       'Gift Store', 'Gourmet Store', 'Grocery Store / Supermarket',
       'Beer Garden', 'Dance Studio', 'Flower Store', 'Garden Center',
       'Carpet and Flooring Contractor',
       'Electric Vehicle Charging Station', 'Retail', 'Martial Arts Dojo',
       'Historic and Protected Site', 'Health and Medicine',
       'Performing Arts Venue', 'Monument', 'General Contractor',
      

In [12]:
restaurants = [i for i in list(la_restaurants["Venue Category"].unique()) if 'Restaurant' in i or 'Burger' in i or 'Pizz' in i or 'Hot Dog' in i or 'BBQ' in i]
code = ""
for i in restaurants:
    code+="(la_restaurants['Venue Category']=='{}')|".format(i)
print(code)

(la_restaurants['Venue Category']=='Sushi Restaurant')|(la_restaurants['Venue Category']=='Latin American Restaurant')|(la_restaurants['Venue Category']=='Fast Food Restaurant')|(la_restaurants['Venue Category']=='Ramen Restaurant')|(la_restaurants['Venue Category']=='Taco Restaurant')|(la_restaurants['Venue Category']=='Restaurant')|(la_restaurants['Venue Category']=='Asian Restaurant')|(la_restaurants['Venue Category']=='Pizzeria')|(la_restaurants['Venue Category']=='Seafood Restaurant')|(la_restaurants['Venue Category']=='Mexican Restaurant')|(la_restaurants['Venue Category']=='Burger Joint')|(la_restaurants['Venue Category']=='Cajun / Creole Restaurant')|(la_restaurants['Venue Category']=='Salad Restaurant')|(la_restaurants['Venue Category']=='Thai Restaurant')|(la_restaurants['Venue Category']=='American Restaurant')|(la_restaurants['Venue Category']=='Italian Restaurant')|(la_restaurants['Venue Category']=='Vegan and Vegetarian Restaurant')|(la_restaurants['Venue Category']=='Chi

In [45]:
la_restaurants = la_restaurants[(la_restaurants['Venue Category']=='Sushi Restaurant')|(la_restaurants['Venue Category']=='Latin American Restaurant')|(la_restaurants['Venue Category']=='Fast Food Restaurant')|(la_restaurants['Venue Category']=='Ramen Restaurant')|(la_restaurants['Venue Category']=='Taco Restaurant')|(la_restaurants['Venue Category']=='Restaurant')|(la_restaurants['Venue Category']=='Asian Restaurant')|(la_restaurants['Venue Category']=='Pizzeria')|(la_restaurants['Venue Category']=='Seafood Restaurant')|(la_restaurants['Venue Category']=='Mexican Restaurant')|(la_restaurants['Venue Category']=='Burger Joint')|(la_restaurants['Venue Category']=='Cajun / Creole Restaurant')|(la_restaurants['Venue Category']=='Salad Restaurant')|(la_restaurants['Venue Category']=='Thai Restaurant')|(la_restaurants['Venue Category']=='American Restaurant')|(la_restaurants['Venue Category']=='Italian Restaurant')|(la_restaurants['Venue Category']=='Vegan and Vegetarian Restaurant')|(la_restaurants['Venue Category']=='Chinese Restaurant')|(la_restaurants['Venue Category']=='Caribbean Restaurant')|(la_restaurants['Venue Category']=='Mediterranean Restaurant')|(la_restaurants['Venue Category']=='Vietnamese Restaurant')|(la_restaurants['Venue Category']=='Hot Dog Joint')|(la_restaurants['Venue Category']=='BBQ Joint')|(la_restaurants['Venue Category']=='Southern / Soul Food Restaurant')|(la_restaurants['Venue Category']=='Greek Restaurant')|(la_restaurants['Venue Category']=='Sandwich Restaurant')|(la_restaurants['Venue Category']=='Indian Restaurant')|(la_restaurants['Venue Category']=='Hotpot Restaurant')|(la_restaurants['Venue Category']=='Japanese Restaurant')|(la_restaurants['Venue Category']=='African Restaurant')|(la_restaurants['Venue Category']=='Ethiopian Restaurant')|(la_restaurants['Venue Category']=='Poke Restaurant')|(la_restaurants['Venue Category']=='Salvadoran Restaurant')|(la_restaurants['Venue Category']=='French Restaurant')|(la_restaurants['Venue Category']=='New American Restaurant')|(la_restaurants['Venue Category']=='Korean Restaurant')|(la_restaurants['Venue Category']=='Peruvian Restaurant')|(la_restaurants['Venue Category']=='Lebanese Restaurant')|(la_restaurants['Venue Category']=='Burrito Restaurant')|(la_restaurants['Venue Category']=='Middle Eastern Restaurant')|(la_restaurants['Venue Category']=='Tex-Mex Restaurant')|(la_restaurants['Venue Category']=='Falafel Restaurant')|(la_restaurants['Venue Category']=='Kosher Restaurant')|(la_restaurants['Venue Category']=='Halal Restaurant')|(la_restaurants['Venue Category']=='Tuscan Restaurant')|(la_restaurants['Venue Category']=='Brazilian Restaurant')|(la_restaurants['Venue Category']=='Filipino Restaurant')|(la_restaurants['Venue Category']=='Caucasian Restaurant')|(la_restaurants['Venue Category']=='Argentinian Restaurant')|(la_restaurants['Venue Category']=='Bangladeshi Restaurant')|(la_restaurants['Venue Category']=='Soup Restaurant')|(la_restaurants['Venue Category']=='Colombian Restaurant')|(la_restaurants['Venue Category']=='Dim Sum Restaurant')|(la_restaurants['Venue Category']=='Cantonese Restaurant')|(la_restaurants['Venue Category']=='Noodle Restaurant')|(la_restaurants['Venue Category']=='Cuban Restaurant')|(la_restaurants['Venue Category']=='Hawaiian Restaurant')|(la_restaurants['Venue Category']=='Armenian Restaurant')|(la_restaurants['Venue Category']=='South American Restaurant')|(la_restaurants['Venue Category']=='Szechuan Restaurant')]

In [14]:
la_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandi,34.034344,-118.299314,Orange Door Sushi,34.032550,-118.299437,Sushi Restaurant
4,Adams-Normandi,34.034344,-118.299314,Sushi Delight,34.032483,-118.299436,Sushi Restaurant
9,Adams-Normandi,34.034344,-118.299314,La Estrella Mexican Food,34.033002,-118.301072,Fast Food Restaurant
17,Adams-Normandi,34.034344,-118.299314,Orange Sekai Ramen,34.032519,-118.299366,Ramen Restaurant
20,Adams-Normandi,34.034344,-118.299314,Lago De Guija,34.032538,-118.298800,Fast Food Restaurant


Una vez hecho esto, llega el paso final, que consiste en combinar estos datos con los de la API de Google Maps para obtener las reseñas de los restaurantes.

In [15]:
!pip install googlemaps

In [16]:
import googlemaps
gmaps = googlemaps.Client(key=API_Key_Google)

In [46]:
lista = []

In [47]:
import numpy as np
from tqdm import tqdm
mean_reviews = []
for restaurant in tqdm(list(la_restaurants.Venue.values)):
    if restaurant not in lista:
        try:
            place_details = gmaps.places(restaurant)
            place = gmaps.place(place_details['results'][0]['place_id'])
            reviews = []
            for i in place['result']['reviews']:
                reviews.append(i["rating"])
            mean_reviews.append(np.mean(reviews))
        except:
            lista.append(restaurant)
            pass
    else:
        pass

100%|██████████| 1284/1284 [16:25<00:00,  1.30it/s]


In [48]:
la_restaurants = la_restaurants[la_restaurants.Venue.isin(lista) == False]

In [49]:
la_restaurants["Mean Review"] = mean_reviews

Sería interesante observar esto en un mapa para cada barrio

In [50]:
from IPython.display import display
for neighborhood in la_restaurants.Neighborhood.unique():
    n = la_restaurants[la_restaurants.Neighborhood == neighborhood]
    n_map = folium.Map(location = [n["Neighborhood Latitude"].values[0],n["Neighborhood Longitude"].values[0]],zoom_start = 16)
    for row in n.iterrows():
        row = row[1]
        name = row["Venue"]+", "+row["Venue Category"]+", "+str(row["Mean Review"])
        folium.Marker(location = [row["Venue Latitude"],row["Venue Longitude"]], popup = name).add_to(n_map)
        n_map.add_child(folium.ClickForMarker(popup=name))
    display(n_map)

El último paso consiste en obtener una matriz de reseñas. Esto nos permitirá, en el siguiente apartado, elaborar un sistema de recomendación de filtarado colaborativo. Para ello, es necesario tener una base de datos donde, para cada *usuario*, en este caso, los barrios, se tiene la informaci
on acerca de qué restaurantes existen en la zona y la nota que tienen. Dado que las notas van de 1 a 5, un 0 indica que ese tipo de restaurante no se encuentra en la zona.

In [51]:
la_onehot = pd.get_dummies(la_restaurants[['Venue Category']], prefix="", prefix_sep="")
la_onehot["Mean Review"] = la_restaurants["Mean Review"]
la_onehot['Neighborhood'] = la_restaurants['Neighborhood']
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]
la_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Armenian Restaurant,Asian Restaurant,BBQ Joint,Bangladeshi Restaurant,Brazilian Restaurant,Burger Joint,...,Southern / Soul Food Restaurant,Sushi Restaurant,Szechuan Restaurant,Taco Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tuscan Restaurant,Vegan and Vegetarian Restaurant,Vietnamese Restaurant,Mean Review
0,Adams-Normandi,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.8
4,Adams-Normandi,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2.2
5,Adams-Normandi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.6
9,Adams-Normandi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.6
17,Adams-Normandi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.2


In [52]:
la_grouped = la_onehot.groupby('Neighborhood').sum().reset_index()
la_grouped

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Armenian Restaurant,Asian Restaurant,BBQ Joint,Bangladeshi Restaurant,Brazilian Restaurant,Burger Joint,...,Southern / Soul Food Restaurant,Sushi Restaurant,Szechuan Restaurant,Taco Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tuscan Restaurant,Vegan and Vegetarian Restaurant,Vietnamese Restaurant,Mean Review
0,Adams-Normandi,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,27.4
1,Angeles Mes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.8
2,Angelino Height,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,22.6
3,Angelus Vist,0,0,0,0,1,0,0,0,2,...,0,0,0,1,0,0,0,0,0,42.8
4,Arlington Height,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Wilshire Cente,0,1,0,0,2,5,0,0,1,...,0,0,0,0,0,1,0,0,0,103.6
145,Wilshire Vist,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17.0
146,Winnetk,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26.8
147,Woodland Hill,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,9.4


In [56]:
df_reviews = pd.DataFrame(columns = ["Neighborhood","Category","Mean Review"]) 
for neighborhood in la_onehot.Neighborhood.unique():
    n_data = la_onehot[la_onehot.Neighborhood == neighborhood]
    for category in la_grouped.columns[1:]:
        n_values = [neighborhood]
        n_values.append(category)
        reviews = la_restaurants[(la_restaurants.Neighborhood == neighborhood)&(la_restaurants["Venue Category"] == category)]["Mean Review"].values
        if la_grouped[la_grouped.Neighborhood == neighborhood][category].values[0] != 0:
            mean_review = np.sum(reviews)/la_grouped[la_grouped.Neighborhood == neighborhood][category].values[0]
        else:
            mean_review = 0
        n_values.append(mean_review)
        df_reviews.loc[len(df_reviews.index)] = n_values
df_reviews = df_reviews[df_reviews["Mean Review"] != 0]
df_reviews.reset_index(inplace = True)
df_reviews.drop("index",axis = 1,inplace = True)
df_reviews

,Neighborhood,Category,Mean Review
0,Adams-Normandi,Fast Food Restaurant,4.3
1,Adams-Normandi,Latin American Restaurant,4.8
2,Adams-Normandi,Ramen Restaurant,4.2
3,Adams-Normandi,Sushi Restaurant,2.5
4,Angelino Height,Asian Restaurant,3.6
...,...,...,...
819,Yucca Corrido,Indian Restaurant,3.4
820,Yucca Corrido,Mexican Restaurant,4.6
821,Yucca Corrido,Pizzeria,4.3
822,Yucca Corrido,Restaurant,5.0


In [60]:
sorted(list(df_reviews.Neighborhood.unique()))

['Adams-Normandi',
 'Angeles Mes',
 'Angelino Height',
 'Angelus Vist',
 'Arlington Height',
 'Arts Distric',
 'Atwater Villag',
 'Baldwin Hill',
 'Baldwin Villag',
 'Beachwood Canyo',
 'Beverly Grov',
 'Beverly Hills Post Offi',
 'Boyle Height',
 'Brentwoo',
 'Brentwood Circl',
 'Bunker Hil',
 'Byzantine-Latino Quarte',
 'Canoga Par',
 'Cartha',
 'Carthay Circl',
 'Castle Height',
 'Central-Alamed',
 'Century Cit',
 'Civic Cente',
 'Country Club Par',
 'Crensha',
 'Crenshaw Mano',
 'Crestvie',
 'Cypress Par',
 'Del Re',
 'Downtown Los Angel',
 'Eagle Roc',
 'East Hollywoo',
 'Echo Par',
 'Edendal',
 'El Seren',
 'Elysian Height',
 'Elysian Valle',
 'Exposition Par',
 'Faircrest Height',
 'Fairfax Distric',
 'Fashion Distric',
 'Financial Distric',
 'Florenc',
 'Franklin Hill',
 'Gallery Ro',
 'Garvanz',
 'Glassell Par',
 'Gramercy Par',
 'Hancock Par',
 'Harbor Cit',
 'Harbor Gatewa',
 'Harvard Height',
 'Harvard Par',
 'Highland Par',
 'Historic Cor',
 'Historic Filipinotow',
 'Histo

Es necesario separar los datos entre el *usuario activo*, que en este caso va a ser el barrio de Los Ágeles Oeste, y el resto de *usuarios*, que son los que nos permiten elbarorar el filtrado colaborativo. Este caso es a modo de ejemplo, y se podría replicar este análisis para cualquier barrio de interés, no solamente para el escogido en esta ocasión. 

In [61]:
inputRest = df_reviews[df_reviews.Neighborhood == "West Los Angel"]
df_reviews = df_reviews[df_reviews.Neighborhood != "West Los Angel"]

In [62]:
inputRest

,Neighborhood,Category,Mean Review
759,West Los Angel,Asian Restaurant,3.633333
760,West Los Angel,BBQ Joint,5.0
761,West Los Angel,Burger Joint,4.733333
762,West Los Angel,Japanese Restaurant,3.8
763,West Los Angel,Korean Restaurant,4.6
764,West Los Angel,Ramen Restaurant,4.2
765,West Los Angel,Restaurant,4.2
766,West Los Angel,Sushi Restaurant,4.0
767,West Los Angel,Szechuan Restaurant,4.8
768,West Los Angel,Taco Restaurant,5.0


## Metodología <a name="metodología"></a>

Antes de elaborar el sistema de recomendación, sería interesante comprender qué es. En este sentido, podemos definirlos como una colección de algoritmos utilizados para sugerir temas a los usuarios, basados en información tomada desde el punto de vista del usuario.

En este caso concreto, la idea es encontrar *usuarios*, en este caso, los vecindarios, que tengan preferencias y opiniones parecidas, en este caso, sobre los tipos de restaurantes, para entonces recomendar items que se hayan parecido al ingreso anterior. Existen diferentes métodos para encontrar los diferentes vecindarios que son similares entre sí. En este caso, el método que vamos a utilizar estará basado en la **Función de Correlación de Pearson**.

Como ya hemos definido anteriormente cuál será el *usuario activo*, quedan por realizar los siguientes pasos:
*   Encuentra a los primeros X barrios similares basándonos en la puntuación de las categorías de los restaurantes.
*   Obtener el registro de la categoria del restaurante que "valoró" el vecindario para cada barrio.
*   Calcular un puntaje de similitud utilizando alguna fórmula.
*   Recomendar las categrías de restaurantes con los puntajes más altos.

Comencemos con el primer paso.

In [63]:
df_group = df_reviews.groupby("Neighborhood")
df_group = sorted(df_group, key = lambda x: len(x[1]), reverse = True)
df_group

[('Sherman Oak',
      Neighborhood                   Category Mean Review
  592  Sherman Oak        American Restaurant         4.4
  593  Sherman Oak                  BBQ Joint         4.2
  594  Sherman Oak               Burger Joint         3.8
  595  Sherman Oak         Chinese Restaurant         3.0
  596  Sherman Oak       Fast Food Restaurant         4.4
  597  Sherman Oak           Greek Restaurant         4.2
  598  Sherman Oak          Indian Restaurant         4.8
  599  Sherman Oak  Middle Eastern Restaurant         4.2
  600  Sherman Oak                   Pizzeria         4.5
  601  Sherman Oak           Ramen Restaurant         5.0
  602  Sherman Oak                 Restaurant         4.8
  603  Sherman Oak           Sushi Restaurant         4.8
  604  Sherman Oak            Thai Restaurant         4.8),
 ('Carthay Circl',
        Neighborhood                  Category Mean Review
  92   Carthay Circl        African Restaurant    4.333333
  93   Carthay Circl            

Una vez hecho esto, es necesario encontrar la similitud entre el barrio escogido y los que forman el grupo, para ello, como ya quedó señalado, se emplea el **Coeficiente de Correlación de Pearson**. Este coeficiente se utiliza para medir la fuerza de una asociación lineal entre dos variables. La fórmula para encontrar este coeficiente entre los conjuntos X e Y con los valores de N se puede ver en la imagen debajo:

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Correlación Pearson")

Los valores brindados por la fórumula puede variar de r = -1 a r = 1, donde 1 se correlaciona directamente entre las dos entidades (esto sería una correlación positiva perfecta) y -1 forma una correlación negativa perfecta.

En nuestro caso, un 1 se refiere a que dos usuarios tiene gustos parecidos, mientras que -1 es lo opuesto.

Ahora, calculemos la Correlación Pearson entre el barrio escogido y el grupo, para almacenarlo en el diccionario, donde la clave es el nombre del barrio y el valor es el coeficiente de correlación.

In [76]:
from math import sqrt
pearsonCorrelationDict = {}

for name,group in df_group:
    group.sort_values(by="Category")
    inputRest.sort_values(by="Category")
    nRatings = len(group)
    temp_df = inputRest[inputRest['Category'].isin(group['Category'].tolist())]
    tempRatingList = temp_df['Mean Review'].tolist()
    tempGroupList = group['Mean Review'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    if Sxx != 0 and Syy != 0 and Sxx*Syy > 0:
        pearsonCorrelationDict[name] = round(Sxy/sqrt(Sxx*Syy),2)
    else:
        pearsonCorrelationDict[name] = 0

In [77]:
pearsonCorrelationDict.items()

dict_items([('Sherman Oak', -0.61), ('Carthay Circl', -0.09), ('Pico-Roberts', 0.48), ('Beverly Grov', 0.17), ('Echo Par', -0.49), ('Wilshire Cente', -0.03), ('Atwater Villag', -0.44), ('Country Club Par', 0.12), ('Edendal', -0.12), ('Jewelry District (Los Angele', -0.51), ('Koreatow', 0.27), ('Little Banglades', -0.21), ('Naud Junction (Los Angele', 0.27), ('Sonorato', -0.03), ('Westwood Villag', 0.22), ('Exposition Par', 0.16), ('Fairfax Distric', -0.06), ('Financial Distric', 0.16), ('Hollywoo', -0.26), ('Lake Balbo', 0.12), ('Little Toky', 0.06), ('Melrose Hil', -0.14), ('Toy Distric', 0.18), ('Victor Height', -0.37), ('Westwoo', 0.32), ('Angelus Vist', 0.25), ('Cartha', 0.2), ('Eagle Roc', -0.32), ('Fashion Distric', -0.42), ('Harbor Cit', -0.37), ('Historic Cor', 0.03), ('Mid-City Wes', 0.03), ('Mid-Wilshir', 0.13), ('Northridg', -0.13), ('Old Bank Distric', 0.25), ('Palm', 0.21), ('Pico-Unio', -0.43), ('University Hill', -0.32), ('West Adams Height', -0.76), ('Yucca Corrido', -0

In [78]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['Neighborhood'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,Neighborhood
0,-0.61,Sherman Oak
1,-0.09,Carthay Circl
2,0.48,Pico-Roberts
3,0.17,Beverly Grov
4,-0.49,Echo Par


In [79]:
topN=pearsonDF.sort_values(by='similarityIndex', ascending=False)
topN.head()

,similarityIndex,Neighborhood
124,1.0,El Seren
129,1.0,Venic
115,1.0,Vermont Squar
125,1.0,Harbor Gatewa
127,1.0,Lafayette Squar


## Análisis <a name="análisis"></a>

Ahora, es el momento de recomendar qué restaurantes deberían abir en función de la categoría. Paraello, tomaremos el peso promedio de los ratings de las categorías de los restaurantes utilizando la Correlación Pearson. Pero para hacer esto, primero necesitamos que losbarrios vean las categorías en nuestro **pearsonDF** a partir del dataframe de puntajes y luego guardar su correlación en una nueva columna llamada \_similarityIndex". Estos se logra juntando estas dos tablas de debajo.

In [80]:
topNRating=topN.merge(df_reviews, on='Neighborhood')
topNRating.head()

,similarityIndex,Neighborhood,Category,Mean Review
0,1.0,El Seren,Mexican Restaurant,5.0
1,1.0,El Seren,Taco Restaurant,4.2
2,1.0,Venic,Fast Food Restaurant,3.4
3,1.0,Venic,Restaurant,3.1
4,1.0,Vermont Squar,Burger Joint,3.8


Ahora todo lo que se necesita hacer es multiplicar el puntaje de la categoría por su peso (El índice de similitud), luego se suman los nuevos puntajes y dividen por la suma de los pesos.

Esto se logra sencillamente multiplicando dos columnas, luego agrupando el dataframe por la columna Category y luego dividiendo dos columnas:

Aqui se muestra la idea de todos los usuarios similares respecto de las categorías de restaurante candidatas para el usuario ingresado:

In [81]:
topNRating['weightedRating'] = topNRating['similarityIndex']*topNRating['Mean Review']
topNRating.head()

,similarityIndex,Neighborhood,Category,Mean Review,weightedRating
0,1.0,El Seren,Mexican Restaurant,5.0,5.0
1,1.0,El Seren,Taco Restaurant,4.2,4.2
2,1.0,Venic,Fast Food Restaurant,3.4,3.4
3,1.0,Venic,Restaurant,3.1,3.1
4,1.0,Vermont Squar,Burger Joint,3.8,3.8


In [82]:
tempTopNRating = topNRating.groupby('Category').agg({'similarityIndex':"sum",'weightedRating':"sum"})
tempTopNRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopNRating.head()

,sum_similarityIndex,sum_weightedRating
Category,,
African Restaurant,0.11,0.501429
American Restaurant,-2.45,-6.070667
Argentinian Restaurant,0.69,3.098000
Armenian Restaurant,0.18,0.756000
Asian Restaurant,-1.60,-0.085400


In [83]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopNRating['sum_weightedRating']/tempTopNRating['sum_similarityIndex']
recommendation_df['Category'] = tempTopNRating.index
recommendation_df.head()

,weighted average recommendation score,Category
Category,,
African Restaurant,4.558442,African Restaurant
American Restaurant,2.477823,American Restaurant
Argentinian Restaurant,4.489855,Argentinian Restaurant
Armenian Restaurant,4.200000,Armenian Restaurant
Asian Restaurant,0.053375,Asian Restaurant


## Resultados y discusión <a name="resultados"></a>

Finalmente, aquí podemos ver las categorías de los restaurantes ordenadas en función de su puntuación.

In [84]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,Category
Category,,
Burger Joint,22.134043,Burger Joint
Indian Restaurant,8.182609,Indian Restaurant
Sushi Restaurant,6.417497,Sushi Restaurant
Taco Restaurant,5.820370,Taco Restaurant
Thai Restaurant,5.456878,Thai Restaurant
Sandwich Restaurant,5.369811,Sandwich Restaurant
Mexican Restaurant,5.291489,Mexican Restaurant
French Restaurant,5.185185,French Restaurant
Restaurant,5.175276,Restaurant


## Conclusión <a name="conclusión"></a>

Como se puede ver, gracias a este sistema de recomendación, una persona interesada en abrir un restaurante en el districto de Los Ángeles Oeste podría llegar a saber qué tipos de restaurante podrían funcionar mejor en esa ubicación, en este caso, una cadena de hamburguesas sería la primera opción.

Este análisis se puede extender a cualquier otro barrio de la ciudad de Los Ángeles, permitiendo así a las personas que quieran abrir un restaurante entender no solamente qué tipos de restaurantes podrían funcionar mejor en función de la localización, sino que también podrían llegar a entender qué localización es la mejor para abir un tipo determinado de restaurante.